In [63]:
using Finch
using SparseArrays
using Metatheory
using Metatheory.EGraphs
using TermInterface
using PrettyPrinting
include("../Source/Spartan.jl")
include("../Source/UtilityFuncs.jl")

uniform_fiber (generic function with 1 method)

In [71]:
is = ["i", "l"]
A = InputTensor("A", ["i", "j"], uniform_fiber([10000, 1000], .01))
B = InputTensor("B", ["j", "l"], uniform_fiber([1000, 10000], .01))
C = InputTensor("C", ["j", "l"], uniform_fiber([1000, 10000], .01))
output_order = ["j", "i"]

verbose = 1
# Sum_i Sum_l B[j,l]*A[i,j]*A[i,j]
output_tensor_1 = spartan(:(Reorder(ReduceDim($+, $is,  MapJoin($*, $B, MapJoin($*, $A, $C))), $output_order)), verbose=verbose, optimize=false)
#println(output_tensor)

global_index_order = ["i", "j", "l"]
# (Sum_l B[j,l])*(Sum_i A[i,j] * A[i,j])
output_tensor_2 = spartan(:(Reorder(ReduceDim($+, $is, MapJoin($*, $B, MapJoin($*, $A, $C))), $output_order)), verbose=verbose,
                             global_index_order = global_index_order, optimize=true)

global_index_order = ["l", "j", "i"]
output_tensor_3 = spartan(:(Reorder(ReduceDim($+, $is, MapJoin($*, $B, MapJoin($*, $A, $C))), $output_order)), verbose=verbose,
                             global_index_order = global_index_order, optimize=true)



global_index_order = ["j", "l", "i"]
output_tensor_4 = spartan(:(Reorder(ReduceDim($+, $is, MapJoin($*, $B, MapJoin($*, $A, $C))), $output_order)), verbose=verbose,
                             global_index_order = global_index_order, optimize=true)


global_index_order = ["i", "l", "j"]
output_tensor_5 = spartan(:(Reorder(ReduceDim($+, $is, MapJoin($*, $B, MapJoin($*, $A, $C))), $output_order)), verbose=verbose,
                            global_index_order = global_index_order, optimize=true)

println("Is Correct?: ", output_tensor_1 == output_tensor_2 && output_tensor_2 == output_tensor_3 && 
                                output_tensor_3 == output_tensor_4 && output_tensor_4 == output_tensor_5)

Expression: 
quote
    ReduceDim(
        +,
        ["i", "l"],
        MapJoin(
            *,
            InputTensor(B, TensorStats(["j", "l"], Dict("j" => 1000, "l" => 10000), 100267.0, 0, Any["i", "j", "l"])),
            MapJoin(
                *,
                InputTensor(A, TensorStats(["i", "j"], Dict("j" => 1000, "i" => 10000), 100255.0, 0, Any["i", "j", "l"])),
                InputTensor(C, TensorStats(["j", "l"], Dict("j" => 1000, "l" => 10000), 99734.0, 0, Any["i", "j", "l"])))))
end
Time to Execute: 5.515714181
Optimized Expression: 
quote
    ReduceDim(
        +,
        ["l"],
        MapJoin(
            *,
            MapJoin(
                *,
                InputTensor(C, TensorStats(["j", "l"], Dict("j" => 1000, "l" => 10000), 99734.0, 0, ["i", "j", "l"])),
                InputTensor(B, TensorStats(["j", "l"], Dict("j" => 1000, "l" => 10000), 100267.0, 0, ["i", "j", "l"]))),
            ReduceDim(
                +,
                ["i"],
                I

In [ ]:
is = ["i", "j"]
B = InputTensor("B", ["i", "k", "l"], uniform_fiber([1000, 1000, 1000], .01))
D = InputTensor("D", ["l", "j"], uniform_fiber([1000, 100], 1.0))
C = InputTensor("C", ["k", "j"], uniform_fiber([1000, 100], 1.0))

verbose = 1
MTTKRP = :(ReduceDim($+, $is, MapJoin($*, $B, MapJoin($C, $D))))
output_tensor_1 = spartan(MTTKRP, optimize=false)
output_tensor_2 = spartan(MTTKRP, optimize=true)




In [35]:
is = ["i", "j"]
X = InputTensor("X", ["i", "j"], uniform_fiber([1000, 1000], .01))
u = InputTensor("u", ["i"], uniform_fiber([1000], 1.0))
v = InputTensor("v", ["j"], uniform_fiber([1000], 1.0))

verbose = 1
# Sum_i Sum_j (X[i,j]+u[i]*v[j])
output_tensor_1 = spartan(:(ReduceDim($+, $is, MapJoin($^, MapJoin($+, $X, MapJoin($*, $u, $v)), 2))), verbose=verbose, optimize=false)

# (Sum_i u[i]^2)*(Sum_j v[j]^2) + (Sum_i u[i] * (Sum_j X[i,j] * V[j])) + (Sum_i u[i] * (Sum_j X[i,j] * V[j])) + (Sum_[i,j] X[i,j]^2)
output_tensor_2 = spartan(:(ReduceDim($+, $is, MapJoin($^, MapJoin($+, $X, MapJoin($*, $u, $v)), 2))), verbose=verbose, optimize=true)

println("Is Correct?: ", output_tensor_1 == output_tensor_2)

Expression: 
quote
    ReduceDim(
        +,
        ["i", "j"],
        MapJoin(
            ^,
            MapJoin(
                +,
                InputTensor(X, TensorStats(["i", "j"], Dict("j" => 1000, "i" => 1000), 10186.0, 0, Any["i", "j"])),
                MapJoin(
                    *,
                    InputTensor(u, TensorStats(["i"], Dict("i" => 1000), 1000.0, 0, Any["i", "j"])),
                    InputTensor(v, TensorStats(["j"], Dict("j" => 1000), 1000.0, 0, Any["i", "j"])))),
            2))
end
Time to Execute: 1.019511778
Optimized Expression: 
quote
    MapJoin(
        +,
        MapJoin(
            +,
            ReduceDim(
                +,
                ["j"],
                MapJoin(
                    *,
                    InputTensor(v, TensorStats(["j"], Dict("j" => 1000), 1000.0, 0, Any["i", "j"])),
                    ReduceDim(
                        +,
                        ["i"],
                        MapJoin(
                         

In [72]:
is = ["i", "j", "k"]
E_fiber = uniform_fiber([1000, 1000], .02)
E_1 = InputTensor("E", ["i", "j"], E_fiber)
E_2 = InputTensor("E", ["j", "k"], E_fiber)
E_3 = InputTensor("E", ["k", "l"], E_fiber)
E_4 = InputTensor("E", ["l", "i"], E_fiber)

verbose = 3
query = :(ReduceDim($+, $is, MapJoin($*, $E_1, MapJoin($*, $E_2, MapJoin($*, $E_3, $E_4)))))
output_1 = spartan(query, verbose=verbose, optimize = false)
output_2 = spartan(query, verbose=verbose, optimize = true)

Expression: 
quote
    ReduceDim(
        +,
        ["i", "j", "k"],
        MapJoin(
            *,
            InputTensor(E, TensorStats(["i", "j"], Dict("j" => 1000, "i" => 1000), 19862.0, 0, Any["i", "j", "k", "l"])),
            MapJoin(
                *,
                InputTensor(E, TensorStats(["j", "k"], Dict("j" => 1000, "k" => 1000), 19862.0, 0, Any["i", "j", "k", "l"])),
                MapJoin(
                    *,
                    InputTensor(E, TensorStats(["k", "l"], Dict("k" => 1000, "l" => 1000), 19862.0, 0, Any["i", "j", "k", "l"])),
                    Reorder(
                        InputTensor(E, TensorStats(["l", "i"], Dict("l" => 1000, "i" => 1000), 19862.0, 0, Any["i", "j", "k", "l"])),
                        ["i", "j", "k", "l"])))))
end
1
2
3
4
5
5
Kernel: ReorderExpr(["i", "j", "k", "l"], InputExpr("A", ["l", "i"], AccessProtocol[t_walk, t_walk], TensorStats(["l", "i"], Dict("l" => 1000, "i" => 1000), 19862.0, 0, Any["i", "j", "k", "l"])))
Output 

SparseHash (0) [1:1000]
├─[1]: 222
├─[2]: 85
│ ⋮
├─[999]: 150
├─[1000]: 119

In [70]:
# Suppose you have a network of power nodes and lines. Further, you need to take weather events/outages into account.
# The following query provides the total available power at each node.
num_nodes = 1000
height = 100
width = 100

# Because we're dealing with positive numbers, the following is true.
Finch.isassociative(::Finch.DefaultAlgebra, ::typeof(min)) = true
Finch.iscommutative(::Finch.DefaultAlgebra, ::typeof(min)) = true
Finch.isannihilator(::Finch.DefaultAlgebra, ::typeof(min), x) = x == 0

transmission_capacity = InputTensor("transmission_capacity", ["i", "j"], uniform_fiber([num_nodes, num_nodes], 5.0/num_nodes))

node_location_data =  uniform_fiber([num_nodes, height, width], 1.0/height/width)
node_location = InputTensor("node_location", ["i", "lat", "long"], node_location_data)
neighbor_node_location = InputTensor("neighbor_node_location", ["j", "lat2", "long2"], node_location_data)

weather_events_data = uniform_fiber([height, width], .01, default_value = 1, non_default_value = 0)
weather_events = InputTensor("weather_events", ["lat", "long"], weather_events_data)
neighbor_weather_events = InputTensor("neighbor_weather_events", ["lat2", "long2"], weather_events_data)

outages_data = uniform_fiber([num_nodes], .1, default_value = 1, non_default_value = 0)
node_outage_percents = InputTensor("node_outage_percents", ["i"], outages_data)
neighbor_node_outage_percents = InputTensor("neighbor_node_outage_percents", ["j"], outages_data)

node_power_data = uniform_fiber([num_nodes], 1.0)
node_power = InputTensor("node_power", ["i"], node_power_data)
neighbor_node_power = InputTensor("neighbor_node_power", ["j"], node_power_data)

available_power_query = :(MapJoin($+, ReduceDim($+, ($["lat", "long"]), MapJoin($*, $node_location, MapJoin($*, $weather_events, 
                                                                           MapJoin($*, $node_outage_percents, $node_power)))),
                                      ReduceDim($+, ($["j"]),
                                            MapJoin($min, ReduceDim($+, $["lat2", "long2"], MapJoin($*, $neighbor_weather_events, 
                                                    MapJoin($*, $neighbor_node_outage_percents, MapJoin($*, $neighbor_node_location, $neighbor_node_power)))), $transmission_capacity))))

verbose = 1
total_neighbor_power_1 = spartan(available_power_query, verbose=verbose, optimize=true)

total_neighbor_power_2 = spartan(available_power_query, verbose=verbose, optimize=false)

println("Is Equivalent: ", total_neighbor_power_1 == total_neighbor_power_2)

Optimized Expression: 
quote
    MapJoin(
        +,
        ReduceDim(
            +,
            ["j"],
            MapJoin(
                min,
                ReduceDim(
                    +,
                    ["lat2", "long2"],
                    MapJoin(
                        *,
                        MapJoin(
                            *,
                            MapJoin(
                                *,
                                InputTensor(neighbor_node_power, TensorStats(["j"], Dict("j" => 1000), 1000.0, 0, Any["i", "j", "lat", "lat2", "long", "long2"])),
                                InputTensor(neighbor_node_location, TensorStats(["j", "lat2", "long2"], Dict("long2" => 100, "j" => 1000, "lat2" => 100), 971.0, 0, Any["i", "j", "lat", "lat2", "long", "long2"]))),
                            InputTensor(neighbor_weather_events, TensorStats(["lat2", "long2"], Dict("long2" => 100, "lat2" => 100), 98.0, 1, Any["i", "j", "lat", "lat2", "long", "long2"]))),
  

In [13]:
countstored(node_location_data)

1013

In [34]:
using Random
I = Finch.fsprand_helper(Random.default_rng(), (100, 100), .01)
V = [rand() for _ in 1:length(I[1])]
fsparse(I,V)

SparseCOO (0.0) [1:100,1:100]
├─├─[8, 1]: 0.5874481201442663
├─├─[21, 1]: 0.8644518849715451
│ ⋮
├─├─[50, 100]: 0.3424241380122627
├─├─[82, 100]: 0.9176587679719371

In [64]:
uniform_fiber([100, 100], .01, non_default_value = 1, default_value = 2)

SparseList (2) [:,1:100]
├─[:,1]: SparseList (2) [1:100]
│ ├─[89]: 1
├─[:,2]: SparseList (2) [1:100]
│ ├─[39]: 1
│ ├─[62]: 1
│ ├─[78]: 1
│ ⋮
├─[:,98]: SparseList (2) [1:100]
│ ├─[24]: 1
├─[:,99]: SparseList (2) [1:100]
│ ├─[52]: 1
│ ├─[81]: 1

In [65]:
uniform_fiber([num_nodes, height, width], 1.0/height/width)

SparseList (0) [:,:,1:100]
├─[:,:,1]: SparseList (0) [:,1:100]
│ ├─[:,5]: SparseList (0) [1:1000]
│ │ ├─[328]: 1
│ ├─[:,16]: SparseList (0) [1:1000]
│ │ ├─[859]: 1
│ │ ⋮
│ ├─[:,93]: SparseList (0) [1:1000]
│ │ ├─[343]: 1
│ │ ├─[508]: 1
│ ├─[:,99]: SparseList (0) [1:1000]
│ │ ├─[338]: 1
├─[:,:,2]: SparseList (0) [:,1:100]
│ ├─[:,1]: SparseList (0) [1:1000]
│ │ ├─[609]: 1
│ ├─[:,3]: SparseList (0) [1:1000]
│ │ ├─[560]: 1
│ │ ⋮
│ ├─[:,91]: SparseList (0) [1:1000]
│ │ ├─[582]: 1
│ ├─[:,92]: SparseList (0) [1:1000]
│ │ ├─[129]: 1
│ ⋮
├─[:,:,99]: SparseList (0) [:,1:100]
│ ├─[:,16]: SparseList (0) [1:1000]
│ │ ├─[249]: 1
│ ├─[:,20]: SparseList (0) [1:1000]
│ │ ├─[26]: 1
│ │ ⋮
│ ├─[:,68]: SparseList (0) [1:1000]
│ │ ├─[928]: 1
│ ├─[:,78]: SparseList (0) [1:1000]
│ │ ├─[341]: 1
├─[:,:,100]: SparseList (0) [:,1:100]
│ ├─[:,20]: SparseList (0) [1:1000]
│ │ ├─[181]: 1
│ ├─[:,38]: SparseList (0) [1:1000]
│ │ ├─[738]: 1
│ │ ⋮
│ ├─[:,75]: SparseList (0) [1:1000]
│ │ ├─[357]: 1
│ ├─[:,84]: SparseList

In [58]:
typeof(Tuple([num_nodes, height, width]))

Tuple{Int64, Int64, Int64}